In [60]:

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from Test_Tools import FactorTest
import Sample_Tools as smpl

import pandas as pd

###########modin#########################
###fit部分没有分布式优势，计算trend时有巨大优势。
# import base.Constants as con
# import modin.pandas as pd
# from distributed.security import Security
# from distributed import Client
# sec = Security(
#     tls_ca_file=con.DC_FILEPATH,
#     tls_client_cert=con.DC_FILEPATH,
#     tls_client_key=con.DK_FILEPATH,
#     require_encryption=True,
# )
# client = Client(f"tls://{con.DASK_HOST}", security=sec)
# from tqdm import tqdm
# from modin.config import ProgressBar
# ProgressBar.enable()
###########modin####################


%load_ext autoreload
%autoreload 2




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
%%time
# data = smpl.get_data(['000001','000004'],end='2021-07-02', gap=1000)
data = smpl.get_sample_by_zs(name='上证50', end='2020-06-29', gap=2500, only_main=True)
datap = pd.DataFrame(data.data.reset_index().set_index('code'))

Wall time: 1.66 s


In [62]:
%%time
def excute_for_multicode(data, func, **pramas):
    return data.groupby(level=1, as_index=False, group_keys=False).apply(func,**pramas)

def kernel(dataframe):
    CLOSE = dataframe.close
    ret = (CLOSE-CLOSE.shift(1))/CLOSE.shift(1)
    res = alpha1(CLOSE, ret)

    return res

def stddev(df, window):
    return df.rolling(window).std()

def ts_argmax(df, window):
    return df.rolling(window).apply(np.argmax) + 1

def rank(df):
    return df.rank(pct=True, axis=0)

def alpha1(close, returns):
    x = close
    
    x[returns < 0] = stddev(returns, 20)
    alpha = rank(ts_argmax(x ** 2, 5))-0.5

    return alpha.fillna(value=0)



Wall time: 0 ns


In [64]:
%%time
# ind_df = excute_for_multicode(data, kernel, **{})
ind_df = datap.groupby(level=0,group_keys=False).apply(kernel)
print(ind_df)

              open        high         low       close        volume  \
code                                                                   
600000   12.287718   12.437467   12.152400   12.295217  8.898013e+05   
600009   30.410669   30.882462   29.988179   30.460789  1.037651e+05   
600016    7.416605    7.506810    7.337339    7.423748  1.448890e+06   
600028    6.178634    6.248962    6.112977    6.183818  1.284257e+06   
600030   16.821828   17.114175   16.574580   16.839919  1.419829e+06   
600031   11.389719   11.593189   11.200631   11.404530  6.669797e+05   
600036   19.031203   19.282900   18.810016   19.046870  7.503687e+05   
600048   11.197154   11.416789   11.005723   11.216375  9.203307e+05   
600050    5.125358    5.209734    5.049759    5.131803  1.647838e+06   
600104   20.890998   21.196690   20.593228   20.900674  2.825476e+05   
600196   21.761944   22.147207   21.401896   21.781326  2.016728e+05   
600276   49.098184   49.910357   48.376349   49.185828  9.542122

In [47]:
datap

,date,open,high,low,close,volume,amount
code,,,,,,,
600000,2010-03-16,20.63,21.20,20.51,21.19,563430.0,1.176060e+09
600009,2010-03-16,17.68,18.26,17.68,18.10,230414.0,4.150379e+08
600016,2010-03-16,7.25,7.38,7.22,7.33,538903.0,3.929420e+08
600028,2010-03-16,11.12,11.35,11.11,11.32,378391.0,4.258330e+08
600030,2010-03-16,27.73,28.23,27.70,28.12,642191.0,1.798765e+09
...,...,...,...,...,...,...,...
601899,2020-06-29,4.37,4.44,4.30,4.37,1917051.0,8.362458e+08
603259,2020-06-29,94.80,97.46,94.29,96.97,179775.0,1.720271e+09
603288,2020-06-29,123.79,125.30,122.62,124.65,50701.0,6.283532e+08
